In [1]:
!pip install torchsummary

In [2]:
# import packages
import os
import pickle
import timm

from tqdm.notebook import tqdm
import random
from tabulate import tabulate
from torchsummary import summary
import albumentations

import cv2
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as t

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchvision import models
import torchvision.transforms.v2 as T

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning

In [3]:
train_df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
with open('/kaggle/input/liver-train-images/conv_range.pkl', 'rb') as f:
    conv_range = pickle.load(f)

In [4]:
train_df['liver_det'] = False

In [5]:
for item in tqdm(conv_range):
    train_df.loc[train_df['patient_id']==int(item),['liver_det']] = True

  0%|          | 0/2717 [00:00<?, ?it/s]

In [6]:
train_df = train_df[train_df['liver_det']==True]

In [7]:
len(train_df[train_df['liver_healthy']==0])

300

In [8]:
train_df1 = train_df[train_df['liver_healthy'] == 0][:250]
test_df1 = train_df[train_df['liver_healthy'] == 0][250:]

In [9]:
train_df2 = train_df[train_df['liver_healthy'] == 1][:750]
test_df2 = train_df[train_df['liver_healthy'] == 1][750:900]

In [10]:
train_df = pd.concat([train_df1,train_df2])
test_df = pd.concat([test_df1,test_df2])

In [11]:
len(test_df)

200

In [12]:
train_dict= {}
for name in tqdm(os.listdir('/kaggle/input/liver-train-images/output/')):
    paths = [f'/kaggle/input/liver-train-images/output/{name}/{item}.png' for item in range(0, 60)]
    train_dict[int(name)] = paths

  0%|          | 0/2717 [00:00<?, ?it/s]

In [13]:
def select_elements_with_spacing(input_list):
    
    selected_indices = []
        
    for i in range(15):
        app = random.randint(i*4,((i+1)*4)-1)
        selected_indices.append(app)
#     selected_indices = [start + (i*5) for i in range(0,12)]
    
    selected_elements = [input_list[index] for index in selected_indices]
    
    return selected_elements

In [14]:
def preprocess_jpeg(jpeg_path):
    
    img = cv2.imread(jpeg_path)
    greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
    
    return greyscale

In [15]:
transforms_train = T.Compose([
                            T.Resize((224, 224), antialias = True),
#                             RandomHorizontalFlip(),  # Randomly flip images left-right
                            T.ColorJitter(brightness=0.2),  # Randomly adjust brightness
                            T.ColorJitter(contrast=0.2),  # Randomly adjust contrast
                            T.RandomAffine(degrees=0, translate=(0.2, 0.25)),
                            T.RandomAffine(degrees=0, shear=10),  # Apply shear transformation
                            T.RandomAffine(degrees=0, scale=(0.9, 1.1)),  # Apply zoom transformation
#                             RandomErasing(p=0.2, scale=(0.02, 0.2)), # Coarse dropout
                            T.ToTensor(),
                        ])

transforms_valid = T.Compose([
                            T.Resize((224, 224), antialias = True),
                            T.ToTensor(),
                        ])


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [16]:
# dataset
class AbdominalData(Dataset):
    """
    Custom dataset class for handling abdominal trauma data classification.
    
    Args:
        df_path (str): Path to the CSV file containing patient labels.
        current_fold (int): The current fold for cross-validation.
        num_fold (int, optional): Total number of folds for cross-validation. Default is 5.
    """
    
    def __init__(self, df,train_dict,transforms):
        
        super().__init__()
        
        # collect all the image instance paths
#         self.img_paths = fetch_img_paths(TRAIN_IMG_PATH)
                
        self.df = df
        self.train_dict = train_dict
        self.transform = transforms

    
    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        
        return len(self.df)
    
    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset by index.
        
        Args:
            idx (int): Index of the dataset to retrieve.
        
        Returns:
            dict: A dictionary containing the image data and labels for different abdominal structures.
        """
        
        # sample 4 image instances
        patient_id = self.df['patient_id'][idx]
        dicom_images = select_elements_with_spacing(self.train_dict[patient_id])
        # add image path in df
        images = []
        
        for d in dicom_images:
            image = preprocess_jpeg(d)
            images.append(image)
            
        images = np.stack(images)
        image = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
        
        image = self.transform(image).squeeze(dim = 1)
        
        label = self.df[self.df.patient_id == int(patient_id)].values[0][1:-2]
        
        liver = np.argmax(label[7:10], keepdims = False)
        
        
        return {
            'image': image,
            'liver': liver,
        }
    
#     def get_splits(self):
#         """
#         Splits the dataset into training and validation subsets based on the current fold.
        
#         Returns:
#             tuple: A tuple containing the training and validation subsets.
#         """
        
#         fold_data = list(self.kf.split(list(self.df['patient_id']))
#         train_indices, val_indices = fold_data[self.current_fold]

#         train_data = self._get_subset(train_indices)
#         val_data = self._get_subset(val_indices)
        
#         return train_data, val_data

#     def _get_subset(self, indices):
#         """
#         Returns a subset of the dataset based on the provided indices.
        
#         Args:
#             indices (list): List of indices to include in the subset.
        
#         Returns:
#             Subset: A subset of the dataset.
#         """
#         return Subset(self, indices)

In [17]:
class MetricsCalculator:
    
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
    
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        """
        Compute the accuracy metric.
        
        Returns:
            float: Accuracy.
        """
        return accuracy_score(self.targets, self.predictions)
    
    def compute_f1(self):
        return f1_score(self.targets, self.predictions, average = None)

    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)

In [18]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        model = timm.create_model('resnet50d', pretrained=False)
        model.conv1 = nn.Sequential(
                nn.Conv2d(15, 32, kernel_size=(3,3), stride=(2,2), padding=(1,1), bias = False),
                nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        model.fc = nn.Sequential(
                nn.Linear(in_features=2048, out_features=1280, bias=True),
                nn.BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.ReLU(inplace=True),
        )
        self.model = model
        self.liver = nn.Linear(1280,3) 
    
    def forward(self, x):
        
        # extract features
        x = self.model(x)
        
        # output logits
        liver = self.liver(x)
        
        return liver

In [19]:
model = CNNModel().to('cuda')

In [20]:
BATCH_SIZE = 64
NUM_EPOCHS = 30
LR = 3e-5

In [21]:
train_df = train_df.drop(['liver_det'],axis=1)
test_df = test_df.drop(['liver_det'],axis=1)

In [22]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
train_data = AbdominalData(train_df,train_dict, transforms_train)
val_data = AbdominalData(test_df,train_dict, transforms_valid)

train_dataloader = DataLoader(train_data,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_data,batch_size = BATCH_SIZE, shuffle = False)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
# bce_b = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([2.0]).to('cuda'))
# bce_e = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([4.0]).to('cuda'))
cce = nn.CrossEntropyLoss(label_smoothing = 0.05, weight = torch.tensor([1.0, 2.0, 4.0]).to('cuda'))
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)

In [24]:
# initialize metrics objects
train_acc_liver = MetricsCalculator('multi')

val_acc_liver = MetricsCalculator('multi')

In [25]:
cd '/kaggle/working/'

/kaggle/working


In [26]:
prev_val_best_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    
    # training
    model.train()
    
    train_loss = 0.0
    val_loss = 0.0
    
    print(f'Epoch: [{epoch+1}/{NUM_EPOCHS}]')
        
    for batch_idx, batch_data in enumerate(tqdm(train_dataloader)):
        
        inputs = batch_data['image'].to('cuda')
        liver = batch_data['liver'].to('cuda')

        
        optimizer.zero_grad()
        l = model(inputs)
        l_loss = cce(l, liver)
        
        total_loss = l_loss 
        total_loss.backward()
        
        optimizer.step()
        
        train_loss += total_loss.item()
        train_acc_liver.update(l, liver)
    
    train_loss = train_loss/(batch_idx+1)
    
    model.eval()
    running_loss = 0.0
    
    for batch_idx, batch_data in enumerate(tqdm(val_dataloader)):
                                                
        inputs = batch_data['image'].to('cuda')
        liver = batch_data['liver'].to('cuda')

        
        l = model(inputs)
        l_loss = cce(l, liver)
        
        total_loss = l_loss 
        
        val_loss += total_loss.item()
        val_acc_liver.update(l, liver)
    
    
    val_loss = val_loss/(batch_idx+1)
    scheduler.step(val_loss)
    
    if val_loss < prev_val_best_loss:
        prev_val_best_loss = val_loss
        print("Validation Loss improved, Saving Model...")
        torch.save(model, f'resnet_50d_15_liver_best.pth')
    if (epoch%6)==0:   
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict':scheduler.state_dict(),
            'train_loss':train_loss,
            'val_loss':val_loss,
            }, f'checkpoint_{epoch}_resnet_50d_15_liver.pth')

    
    
    metrics_data = [
                    ["liver", 
                        train_acc_liver.compute_accuracy(),
                        val_acc_liver.compute_accuracy(),
                        train_acc_liver.compute_auc(),
                        val_acc_liver.compute_auc(),
                        train_acc_liver.compute_f1(),
                        val_acc_liver.compute_f1()]
                ]
    
    # verbose
    print('')
    print(tabulate(metrics_data, headers=["", "Train Acc", "Val Acc", "Train AUC", "Val AUC", "Train F1", "Val F1"]))
    
    print(f'\nMean Train Loss: {train_loss:.3f}')
    print(f'Mean Val Loss: {val_loss:.3f}\n')
    
    #reset metrics
    train_acc_liver.reset()
    val_acc_liver.reset()


Epoch: [1/30]


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Validation Loss improved, Saving Model...

         Train Acc    Val Acc    Train AUC    Val AUC  Train F1                            Val F1
-----  -----------  ---------  -----------  ---------  ----------------------------------  ----------------------------------
liver        0.471      0.335     0.553101   0.506528  [0.61962693 0.28063241 0.13793103]  [0.38647343 0.29508197 0.        ]

Mean Train Loss: 1.133
Mean Val Loss: 1.133

Epoch: [2/30]


  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]


         Train Acc    Val Acc    Train AUC    Val AUC  Train F1                            Val F1
-----  -----------  ---------  -----------  ---------  ----------------------------------  ----------------------------------
liver        0.838       0.73     0.942827   0.665389  [0.89678284 0.63265306 0.77586207]  [0.83495146 0.4109589  0.22222222]

Mean Train Loss: 0.702
Mean Val Loss: 0.849

Epoch: [30/30]


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]


         Train Acc    Val Acc    Train AUC    Val AUC  Train F1                            Val F1
-----  -----------  ---------  -----------  ---------  ----------------------------------  ----------------------------------
liver        0.825      0.685     0.930293   0.660806  [0.89378758 0.60574413 0.66666667]  [0.79298246 0.46       0.13333333]

Mean Train Loss: 0.718
Mean Val Loss: 0.906

